This file useses the purchases of only the 2021 FA, because we need prod. char

In [298]:
%reset -f

In [304]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pandasgui import show
import socket
import numpy as np
from unidecode import unidecode
import unicodedata
import re
from difflib import SequenceMatcher
import pyblp


Functions

In [305]:
def calculate_market_shares_with_outside_good(df_transac, outside_adjustment = 2, max = 0.4):
    """
    Calculate market shares using market-specific N_i from the DataFrame

    outside_adjustment: divides all the outside shares by that number to decrease them 
    max: puts ceiling on the outside shares
    """
    # Group by product (j) and market (t) to count purchases
    product_counts = df_transac.groupby(['product_ids', 'market_ids']).size().reset_index(name='purchases')
    
    # Get N_i for each market (taking first occurrence since it's constant within market)
    market_populations = df_transac.groupby('market_ids')['N_i'].first().reset_index()
  
    # Merge N_i with product counts
    product_counts = product_counts.merge(market_populations, on='market_ids')
    
    # Calculate market shares
    product_counts['shares'] = product_counts['purchases'] / product_counts['N_i']
    product_counts['shares_real'] =product_counts['purchases'] / product_counts['N_i']
                                               
    x_vars = [col for col in df_transac.columns if col.startswith('x')] #list of variables starting with x 
    
    # Get product characteristics
    product_chars = df_transac.groupby(['product_ids', 'market_ids']).agg(
        {var: 'first' for var in x_vars} | {'prices': 'first', 'model_id': 'first'}
    ).reset_index()
    
    # Merge market shares with product characteristics
    final_df = product_chars.merge(product_counts[['product_ids', 'market_ids', 'shares', 'shares_real', 'N_i']], on=['product_ids', 'market_ids'])
    
    # Calculate outside good share for each market
    market_shares = final_df.groupby('market_ids')['shares'].sum()
    outside_shares = 1 - market_shares
    outside_shares = outside_shares / outside_adjustment
    outside_shares = np.minimum(outside_shares, max)
    market_shares_real = final_df.groupby('market_ids')['shares_real'].sum()
    outside_shares_real = 1 - market_shares_real
    
    
    # Create outside good rows
    markets = df_transac['market_ids'].unique()
    outside_goods = pd.DataFrame()
    outside_goods['market_ids'] = markets
    outside_goods['product_ids'] = 0

    for var in x_vars:
        if pd.api.types.is_numeric_dtype(df_transac[var]):
            outside_goods[var] = 0  # Set numeric variables to 0
        else:
            outside_goods[var] = '-'  # Set categorical variables to '-'
    
    outside_goods['prices'] = 0
    outside_goods['shares'] = [outside_shares[t] for t in markets]
    outside_goods['shares_real'] = [outside_shares_real[t] for t in markets]

    # Combine outside goods with other products
    final_df = pd.concat([final_df, outside_goods])
    
    # Sort by market and product ID
    final_df = final_df.sort_values(['market_ids', 'product_ids'])
    
    return final_df



In [328]:
car_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transac_with_chars.csv')
print(os.path.abspath(car_path))
df = pd.read_csv(car_path, encoding = 'latin1')

c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\interm_data\yearly_data\Transacciones\transac_with_chars.csv


## data cleaning

1. select vars that could be relevant
2. rename prod. char as x1, x2.. and do other renaming 
3.  to reduce fixed effects if a variable is categorical and has more than 5 values we remove it. 

In [329]:
#### 1. variable selection 
char_list  = ["Motor - Combustible", "Motor - Cilindrada",
    "TransmisiÃ³n y chasis - Motor - tracciÃ³n", "TransmisiÃ³n y chasis - TransmisiÃ³n",
    "TransmisiÃ³n y chasis - SuspensiÃ³n trasera", "Medidas y capacidades - Largo",
    "Medidas y capacidades - Alto", "Medidas y capacidades - Distancia entre ejes",
    "Confort - Aire acondicionado", "Confort - Tapizados", "Confort - Cierre de puertas", 
    "Confort - Vidrios (del. - tras.)", "Confort - Espejos exteriores", "Confort - Espejo interior",
    "Confort - Faros delanteros", "Confort - Faros antiniebla", "Confort - Computadora de a bordo",
    "Confort - DirecciÃ³n asistida", "Seguridad - Airbags", "Tipo de Producto"]

model_vars = ["IDProductoCM",  "Modelo",  "Marca", 'simplified_model',
            "Precio Unitario", "Rut Unidad de Compra", "RegiÃ³n Unidad de Compra", "Sector",
            "year_df", "Nro LicitaciÃ³n PÃºblica", "Fecha EnvÃ­o OC", "Cantidad", "Rut Proveedor",
             "Nombre Proveedor Sucursal", ]
df = df[model_vars + char_list]


### 2. rename vars 
for n, col in enumerate(char_list, start=1):  # Start numbering from 1
    df.rename(columns={col: f'x{n}'}, inplace=True)

df = df.rename(columns={
    'Fecha EnvÃ­o OC': 'Fecha',
    'RegiÃ³n Unidad de Compra': 'Region', 
    'IDProductoCM': 'product_ids',
})

### 3. drop vars to reduce number of parameters to estimate
for n, col in enumerate(char_list, start=1):  # Start numbering from 1
    print(f"number of unique values in {f'x{n}'}: {df[f'x{n}'].nunique()}")


drop_list = ['x4', 'x5', 'x10', 'x12', 'x13','x15',  'x16', 'x17', 'x18', 'x19']
df = df.drop(drop_list, axis=1)

number of unique values in x1: 4
number of unique values in x2: 65
number of unique values in x3: 3
number of unique values in x4: 17
number of unique values in x5: 39
number of unique values in x6: 132
number of unique values in x7: 119
number of unique values in x8: 93
number of unique values in x9: 5
number of unique values in x10: 8
number of unique values in x11: 4
number of unique values in x12: 8
number of unique values in x13: 21
number of unique values in x14: 2
number of unique values in x15: 39
number of unique values in x16: 6
number of unique values in x17: 1
number of unique values in x18: 7
number of unique values in x19: 12
number of unique values in x20: 8


1. Create month-year 
2. create markets and number of consumers
3. create consumer group

In [330]:
#1. create month-year 
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df['month'] = df['Fecha'].dt.month
df['year'] = df['Fecha'].dt.year
df.drop('Fecha', axis=1, inplace=True)

#2. define the market and the number of consumers in the market 
df['semester'] = ((df['month'] - 1) // 6) + 1
df['market_ids'] = df.groupby(['year', 'semester', 'Region']).ngroup()
print(df['market_ids'].nunique())

region_counts = df.groupby('Region')['Rut Proveedor'].nunique()
df['N_i'] = df['Region'].map(region_counts)

## 3. create consumer groups based on zone-sector
#print(df['Region'].unique())
Norte = ['Arica y Parinacota', 'TarapacÃ\x83Â\x83Ã\x82Â¡', 'Antofagasta', 'Atacama', 'Coquimbo']
Centro = ["Lib. Gral. Bdo. O'Higgins", 'Maule', 'Metropolitana', 'ValparaÃ\x83Â\x83Ã\x82Â\xadso' ]
Sur = ['BÃ\x83Â\x83Ã\x82Â\xado-BÃ\x83Â\x83Ã\x82Â\xado', 'Los RÃ\x83Â\x83Ã\x82Â\xados', 'Ã\x83Â\x83Ã\x82Â\x91uble',
    'Los Lagos', 'AraucanÃ\x83Â\x83Ã\x82Â\xada', 'Magallanes y AntÃ\x83Â\x83Ã\x82Â¡rtica', 'AysÃ\x83Â\x83Ã\x82Â©n']


def get_zone(region):
    if region in Norte:
        return 'Norte'
    elif region in Centro:
        return 'Centro'
    elif region in Sur:
        return 'Sur'
    else:
        return 'Other'  # For any regions not in the lists
df['Zona'] = df['Region'].apply(get_zone)
df['k2'] = df.groupby(['Sector', 'Zona']).ngroup()
df['k'] = df.groupby('Sector').ngroup()


217


0. generate unique identifier by simplified model 
1. convert to numeric values and drop chars with NaNs 
2. drop columns with NaNs since they can not be used.
3. prices to numeric

In [331]:
#0. 
# Create a dictionary mapping unique models to IDs
model_to_id = {model: idx for idx, model in enumerate(df['simplified_model'].unique())}

# Create new column with the numeric IDs
df['model_id'] = df['simplified_model'].map(model_to_id)


# 1. convert to numeric values and drop chars with NaNs 
def extract_number(value):
    if pd.isna(value) or value == 'N/D':
        return np.nan
    # Extract numbers using string methods
    if isinstance(value, str):
        # Find first sequence of numbers (including decimals)
        numbers = ''.join(c for c in value if c.isdigit() or c == '.')
        return float(numbers) if numbers else value
    return value

for column in df.columns:
    try:
        df[column] = df[column].apply(extract_number)
    except:
        continue # If conversion fails, keep the original values


## 2. drop rows with NaNs
x_columns = [col for col in df.columns if col.startswith('x')]
x_columns_with_missing = [col for col in x_columns if df[col].isna().any()] #cols start with x and have NaNs
non_x_columns = [col for col in df.columns if not col.startswith('x')] # Get non-x columns 
df = df.drop(columns=x_columns_with_missing) # Drop x-columns with missing values

remaining_x_columns = [col for col in df.columns if col.startswith('x')]
new_x_names = [f'x{i+1}' for i in range(len(remaining_x_columns))] #rename x cols so that they dont skip numbers 
rename_dict = dict(zip(remaining_x_columns, new_x_names))
df = df.rename(columns=rename_dict) # Rename only the x-columns

# 3. prices to numeric 
df['prices'] = df['Precio Unitario'].replace(r',.*', '', regex=True) #remove commas from numbers
columns_to_convert = ['prices'] 
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

only keep product categories with more than 100 obs 

In [332]:
print(df.value_counts('x6'))
# Look at value counts and drop categories with less than 100 occurrences
x6_counts = df['x6'].value_counts()
print("Categories and their counts:")
for cat, count in x6_counts.items():
    print(f"{cat}: {count}")

# Keep only categories with >= 100 occurrences 
valid_categories = x6_counts[x6_counts >= 100].index
df['x6'] = df['x6'].apply(lambda x: x if x in valid_categories else None)



x6
CAMIONETA          2183
SUV                 894
MINIBUS             364
SEDÃN              232
CAMIÃN LIVIANO      97
FURGÃN              88
HATCHBACK            35
CARGO                 1
Name: count, dtype: int64
Categories and their counts:
CAMIONETA: 2183
SUV: 894
MINIBUS: 364
SEDÃN: 232
CAMIÃN LIVIANO: 97
FURGÃN: 88
HATCHBACK: 35
CARGO: 1


In [333]:
product_data = calculate_market_shares_with_outside_good(df, outside_adjustment= 6, max = .3)
average_shares = product_data.loc[product_data["product_ids"] != 0, "shares"].mean()
avg_outside  = product_data.loc[product_data["product_ids"] == 0, "shares"].mean()

print(average_shares, avg_outside)


0.054347632399025965 0.053171111756565574


check that shares 1. are positive and 2. sum upp to less than 1. 

In [334]:
###1. Check for negative shares
# percentage of neg shares. (if share is negative replace by 2%)
negative_shares_percentage = (product_data['shares'] < 0).mean() * 100
print(f"Percentage of negative shares: {negative_shares_percentage:.2f}%")
product_data['shares'] = product_data['shares'].apply(lambda x: 0.02 if x < 0.02 else x)

#shares have to sum up to 1, normalize  by market_ids
product_data['shares_sum'] = product_data.groupby('market_ids')['shares'].transform('sum')
product_data['shares'] = product_data['shares'] / (product_data['shares_sum']+0.01) # add 0.01 since shares have to sum up to less than 1. 

# 3. check that outside shares are not too high and that there are no negative shares
outside_good_shares = product_data[product_data['product_ids'] == 0]['shares'].describe()
print(outside_good_shares)
negative_shares_percentage = (product_data['shares'] < 0).mean() * 100
print(f"Percentage of negative shares: {negative_shares_percentage:.2f}%")

# 4. check that no share sum up to more than 1 
sum_market_shares = product_data.groupby('market_ids')['shares'].sum()
markets_with_high_shares = sum_market_shares[(sum_market_shares > 1) ]
print(markets_with_high_shares)

average_shares = product_data.loc[product_data["product_ids"] != 0, "shares"].mean()
print(average_shares)

Percentage of negative shares: 1.33%
count    217.000000
mean       0.210553
std        0.206621
min        0.004178
25%        0.024722
50%        0.150289
75%        0.338553
max        0.761905
Name: shares, dtype: float64
Percentage of negative shares: 0.00%
Series([], Name: shares, dtype: float64)
0.061444957706533715


## create optimal instruments and check their relevance

In [335]:
def calculate_instrument(group, var):
    " share of observations with the same 'var' within the market. "
    total_count = len(group)
    return group[var].apply(lambda x: (group[var] == x).sum() - 1) / (total_count - 1 if total_count > 1 else 1)

 

# Assuming df is your dataframe
def avg_others(group, var ):
    return group[var].apply(lambda x: (group[var].sum() - x) / (len(group) - 1 if len(group) > 1 else 1))


def calculate_demand_instruments(df, var, start_index):
    """Calculate demand instruments for each unique value of 'var' across the entire DataFrame."""
    # Make a copy
    result_df = df.copy()   
    
    # All unique values of 'var' in the entire dataset
    all_unique_values = [val for val in df[var].unique() if val != '-']
    
    # For each unique value, create one column with the share in each market
    for i, value in enumerate(all_unique_values):
        col_name = f'demand_instruments{start_index + i}'
        
        # Share of rows in each market_ids group whose `var` equals `value`
        # groupby + transform keeps the same shape & index alignment
        result_df[col_name] = (
            df
            .groupby('market_ids')[var]
            .transform(lambda x: (x == value).sum() / len(x))
        )
        
    return result_df
 
# share that has the same x6 and x1 value as the product in question
product_data['demand_instruments0'] = product_data.groupby('market_ids', group_keys=False).apply(calculate_instrument, var='x1')
#product_data['demand_instruments1'] = product_data.groupby('market_ids', group_keys=False).apply(calculate_instrument, var='x6')


# average of x2 and x3 among the competitors products 
product_data['demand_instruments1'] = product_data.groupby('market_ids', group_keys=False).apply(avg_others, var='x2')
product_data['demand_instruments2'] = product_data.groupby('market_ids', group_keys=False).apply(avg_others, var='x3')

product_data = calculate_demand_instruments(product_data, var='x6', start_index=3)

instrument_cols = [col for col in product_data.columns 
                   if col.startswith('demand_instruments')]
corr_with_price = product_data[instrument_cols].corrwith(product_data['prices'])
print(corr_with_price)

from scipy.stats import pearsonr
import statsmodels.api as sm

results = []
for col in instrument_cols:
    # Pearson’s correlation returns (correlation_coefficient, p_value)
    corr_value, p_value = pearsonr(product_data[col], product_data['prices'])
    results.append({
        'instrument': col,
        'correlation': corr_value,
        'p_value': p_value
    })

# 3. Convert results into a DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Dependent variable (y)
y = product_data['prices']

# Independent variables (X) - the instrument columns
X = product_data[instrument_cols]

# Add a constant term to the regression
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

df_store = df.copy()
product_data_store = product_data.copy() # to have a copy of the original data

demand_instruments0    0.237613
demand_instruments1   -0.253432
demand_instruments2   -0.262644
demand_instruments3    0.012428
demand_instruments4    0.187580
demand_instruments5         NaN
demand_instruments6   -0.128032
demand_instruments7   -0.080548
dtype: float64
            instrument  correlation       p_value
0  demand_instruments0     0.237613  5.798280e-39
1  demand_instruments1    -0.253432  2.944032e-44
2  demand_instruments2    -0.262644  1.615889e-47
3  demand_instruments3     0.012428  5.008508e-01
4  demand_instruments4     0.187580  1.173777e-24
5  demand_instruments5          NaN           NaN
6  demand_instruments6    -0.128032  3.328351e-12
7  demand_instruments7    -0.080548  1.243996e-05
                            OLS Regression Results                            
Dep. Variable:                 prices   R-squared:                       0.250
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-

C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



## aggregate BLP


In [347]:
df = df_store.copy()
product_data = product_data_store.copy() # to have a copy of the original data
 
df = df.dropna(subset=['x6'])
product_data = product_data.dropna(subset=['x6'])
product_data.drop(columns=['demand_instruments5'], inplace=True)

In [346]:
show(product_data)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [348]:
X1_formulation = pyblp.Formulation('0 + prices + x1 + x2 + x4 + x5 + x6', absorb='')
X1_formulation = pyblp.Formulation('0 + prices   + x2', absorb='') # to avoid the error 'factors contain missing values' 

X2_formulation = pyblp.Formulation('1 + prices + x6')
product_formulations = (X1_formulation, X2_formulation)
 
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})

mc_problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration)
opti_method = pyblp.Optimization('bfgs', {'gtol': 1e-4}) # 'bfgs' ignores the bounds 
opti_method = pyblp.Optimization('tnc', {'gtol': 1e-4}) # 'knitro' respects the bounds

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N      I     K1    K2    MD 
---  ----  -----  ----  ----  ----
217  2752  10850   2     6     6  

Formulations:
       Column Indices:           0       1            2               3               4             5    
-----------------------------  ------  ------  ---------------  -------------  ---------------  ---------
 X1: Linear Characteristics    prices    x2                                                              
X2: Nonlinear Characteristics    1     prices  x6['CAMIONETA']  x6['MINIBUS']  x6['SEDÃ\x81N']  x6['SUV']


In [ ]:
K2 = 6
# Build full k x k arrays for lb and ub
lb_sigma = np.zeros((K2, K2))
ub_sigma = np.zeros((K2, K2))

for i in range(K2):
    for j in range(K2):
        if i == j:
            # Diagonal: allow >= 0, no upper bound.
            lb_sigma[i, j] = 0.0
            # pyblp treats None as +∞
            ub_sigma[i, j] = 15 
        else:
            # Off-diagonal forced to 0
            lb_sigma[i, j] = 0.0
            ub_sigma[i, j] = 0.0


results = mc_problem.solve(sigma=np.eye(K2),
                            sigma_bounds= (lb_sigma, ub_sigma), 
                            optimization=opti_method)
results

Solving the problem ...

Nonlinear Coefficient Initial Values:
    Sigma:             1           prices      x6['CAMIONETA']  x6['MINIBUS']  x6['SEDÃ\x81N']    x6['SUV']  
---------------  -------------  -------------  ---------------  -------------  ---------------  -------------
       1         +1.000000E+00                                                                               
    prices       +0.000000E+00  +1.000000E+00                                                                
x6['CAMIONETA']  +0.000000E+00  +0.000000E+00   +1.000000E+00                                                
 x6['MINIBUS']   +0.000000E+00  +0.000000E+00   +0.000000E+00   +1.000000E+00                                
x6['SEDÃ\x81N']  +0.000000E+00  +0.000000E+00   +0.000000E+00   +0.000000E+00   +1.000000E+00                
   x6['SUV']     +0.000000E+00  +0.000000E+00   +0.000000E+00   +0.000000E+00   +0.000000E+00   +1.000000E+00

Nonlinear Coefficient Lower Bounds:
    Sigma:          

The model may be under-identified. The total number of unfixed parameters is 8, which is more than the total number of moments, 6. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.



At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                                                                            
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value    

In [341]:
opt_instr = results.compute_optimal_instruments()
prb_opt = opt_instr.to_problem() 
results_opt = prb_opt.solve(sigma=np.eye(K2),
                            sigma_bounds= (lb_sigma, ub_sigma), 
                            optimization=opti_method)
results_opt

AttributeError: 'list' object has no attribute 'compute_optimal_instruments'

## Py-BLP with micro-moments. 

product_data is the data with the prices/shares/prod char,  in the documentation called product data
agent_data is the data with the demographics, in our case it has the distribution of demographics. 

\begin{align*}
    &E[\text{i purchases new vehicle} \mid \{\bar{y}_i < \bar{y}_1\}], \\
    &E[\text{i purchases new vehicle} \mid \{\bar{y}_1 \leq \bar{y}_i < \bar{y}_2\}], \\
    &E[\text{i purchases new vehicle} \mid \{\bar{y}_i \geq \bar{y}_2\}];
\end{align*}

\begin{align*}
    &E[f_{si} \mid \{\text{i purchases a minivan}\}], \\
    &E[f_{si} \mid \{\text{i purchases a station wagon}\}], \\
    &E[f_{si} \mid \{\text{i purchases a sport-utility}\}], \\
    &E[f_{si} \mid \{\text{i purchases a full-size van}\}].
\end{align*}


For each market create one obs. for each local agency in the market. 

### Loading data

1. product_data needs to have instruments explicitly defined as demand_instruments{i} variables? I am not sure because chat gpt at some pooint said that X1 variables automatically become instruments. 

In [293]:
df = df_store.copy()
product_data = product_data_store.copy() # to have a copy of the original data
 
df = df.dropna(subset=['x6'])
product_data = product_data.dropna(subset=['x6'])

In [294]:
# Generate dummy variables
dummies = pd.get_dummies(product_data['x6'])

product_data.drop(columns=['x6'], inplace=True)

# Rename the columns of the dummies to sequential names (x6, x7, ...)
dummies.columns = [f'x{i}' for i in range(6, 6 + dummies.shape[1])]

# Concatenate the dummies with the original DataFrame
product_data = pd.concat([product_data, dummies], axis=1)
#show(product_data)

create product_data instrumetns 

In [16]:
import pandas as pd
import numpy as np

def create_competition_variables(df):
    # Create empty columns for the new variables
    df['demand_instruments_0'] = 0.0
    df['demand_instruments_1'] = 0.0
    df['demand_instruments_2'] = 0.0
    df['demand_instruments_3'] = 0.0
    
    # Iterate through each market
    for market_id in df['market_ids'].unique():
        # Get data for current market, excluding outside option
        market_data = df[(df['market_ids'] == market_id) & (df['product_ids'] != 0)].copy()
        
        # For each product in the market
        for product_id in market_data['product_ids']:
            # Get rivals' data (excluding current product)
            rivals = market_data[market_data['product_ids'] != product_id]
            
            # 1. Share of rivals with 'climatizador'
            climatizador_share = (rivals['x4'].str.startswith('climatizador', na=False).sum() 
                                / len(rivals) if len(rivals) > 0 else 0)
            
            # 2. Average price of rivals
            avg_rival_price = rivals['prices'].mean() if len(rivals) > 0 else 0
            
            # 3. Average x2 of rivals
            avg_rival_x2 = rivals['x2'].mean() if len(rivals) > 0 else 0
            
            # 4. Average x3 of rivals
            avg_rival_x3 = rivals['x3'].mean() if len(rivals) > 0 else 0
            
            # Assign values to the current product
            mask = (df['market_ids'] == market_id) & (df['product_ids'] == product_id)
            df.loc[mask, 'demand_instruments_1'] = climatizador_share
            df.loc[mask, 'demand_instruments_0'] = avg_rival_price
            df.loc[mask, 'demand_instruments_2'] = avg_rival_x2
            df.loc[mask, 'demand_instruments_3'] = avg_rival_x3
    
    return df

# Apply the function to your dataframe
product_data = create_competition_variables(product_data)
#product_data.drop(columns=['demand_instruments_1', 'demand_instruments_2', 'demand_instruments_3', ], inplace=True)


In [17]:
# Step 1: Map each market_id to its corresponding Region
market_region_df = df[['market_ids', 'Region']].drop_duplicates()

# Step 2: Count the number of observations for each region-sector pair
region_sector_counts = df.groupby(['Region', 'Sector', 'k']).size().reset_index(name='count')

# Step 3: Merge the counts into a combined dataset for each market_id and its region
agent_data = (
    market_region_df
    .merge(region_sector_counts, on='Region', how='left')
)

agent_data = agent_data.loc[agent_data.index.repeat(agent_data['count'])]
agent_data = agent_data.reset_index(drop=True)


unique_k = sorted(agent_data['k'].unique()) #list of different k values

dummy_vars = []
for i, k_value in enumerate(unique_k, 1):
    
    var_name = f'a{i}'
    agent_data[var_name] = (agent_data['k'] == k_value).astype(int)
    dummy_vars.append(var_name)
    
# Create the formula string
dummy_vars
agent_formulation_string =' + '.join(dummy_vars)
agent_data.drop(['Region', 'Sector'], axis=1, inplace=True)

In [18]:
agent_data['N'] = agent_data['market_ids'].value_counts()[agent_data['market_ids']].values
agent_data = agent_data.drop('count', axis=1)
agent_data['weights'] = 1 / agent_data['N']
m = 15 
for i in range(0, m ): #important: nodes have to start from 0
    agent_data[f'nodes{i}'] = np.random.chisquare(df=3, size=len(agent_data))

In [19]:
product_data

,product_ids,market_ids,x1,x2,x3,x4,x5,prices,model_id,shares,...,shares_sum,x6,x7,x8,x9,x10,demand_instruments_0,demand_instruments_1,demand_instruments_2,demand_instruments_3
15,0,0,-,0.0,0.0,-,-,0.0,NaN,0.544218,...,1.000000,True,False,False,False,False,0.000000e+00,0.000000,0.000000,0.000000
111,1537010,0,delantero - integral,5253.0,1857.0,climatizador bizona,antideslumbrante manual,23249664.0,44.0,0.068027,...,1.000000,False,False,False,False,True,1.697197e+07,0.800000,5356.000000,1881.000000
140,1537046,0,delantero - integral,5335.0,1815.0,climatizador bizona,antideslumbrante manual,14654600.0,6.0,0.068027,...,1.000000,False,True,False,False,False,1.869099e+07,0.800000,5339.600000,1889.400000
149,1537048,0,delantero - integral,5335.0,1815.0,climatizador bizona,antideslumbrante manual,18958060.0,6.0,0.068027,...,1.000000,False,True,False,False,False,1.783030e+07,0.800000,5339.600000,1889.400000
218,1538481,0,delantero - integral,5732.0,1923.0,climatizador trizona,antideslumbrante automÃ¡tico,16017995.0,14.0,0.068027,...,1.000000,False,True,False,False,False,1.841831e+07,0.800000,5260.200000,1867.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,2043071,216,delantero - integral,5335.0,1815.0,climatizador bizona,antideslumbrante manual,23958655.0,6.0,0.105374,...,1.305714,False,True,False,False,False,2.216525e+07,0.428571,5200.000000,1925.142857
2581,2044390,216,delantero - delantera,5253.0,1990.0,manual,antideslumbrante manual,27689076.0,154.0,0.105374,...,1.305714,False,False,True,False,False,2.163233e+07,0.571429,5211.714286,1900.142857
2646,2048605,216,delantero - delantera,5425.0,1882.0,manual,antideslumbrante manual,22162800.0,17.0,0.105374,...,1.305714,False,True,False,False,False,2.242180e+07,0.571429,5187.142857,1915.571429
2663,2049344,216,delantero - delantera,5546.0,2555.0,manual,antideslumbrante manual,24635700.0,174.0,0.105374,...,1.305714,False,False,False,False,False,2.206853e+07,0.571429,5169.857143,1819.428571


In [20]:
agent_data

,market_ids,k,a1,a2,a3,a4,a5,a6,a7,N,...,nodes5,nodes6,nodes7,nodes8,nodes9,nodes10,nodes11,nodes12,nodes13,nodes14
0,12,0,1,0,0,0,0,0,0,1200,...,9.144960,2.164913,2.742728,0.411056,5.377454,0.996362,2.323817,1.500411,0.408366,0.161630
1,12,0,1,0,0,0,0,0,0,1200,...,11.306754,19.871454,2.349071,8.416804,2.627442,1.070439,1.459562,0.847405,4.507280,2.360715
2,12,0,1,0,0,0,0,0,0,1200,...,1.224990,0.801937,2.037401,1.700438,1.467424,2.382328,2.148180,2.021020,2.954609,4.285607
3,12,0,1,0,0,0,0,0,0,1200,...,4.756356,5.110103,0.271105,2.540345,3.613013,4.144346,0.483197,2.711053,0.358984,5.419879
4,12,0,1,0,0,0,0,0,0,1200,...,0.418697,0.941283,2.191737,0.907711,3.321904,4.424199,3.021464,0.841830,6.190847,1.287114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52871,216,4,0,0,0,0,1,0,0,65,...,0.438364,0.454442,2.609159,2.893024,0.062987,8.300748,0.275669,2.703512,2.799947,0.557371
52872,216,4,0,0,0,0,1,0,0,65,...,2.817884,4.169082,0.631015,2.399733,1.303227,1.864653,1.316167,0.759356,0.719655,4.257006
52873,216,5,0,0,0,0,0,1,0,65,...,0.219610,5.233705,1.832999,2.033618,0.071981,1.125377,2.019622,4.193413,2.564793,3.023404
52874,216,6,0,0,0,0,0,0,1,65,...,0.618353,3.417940,0.468662,6.107124,2.067359,2.721539,1.480029,3.798709,4.823185,2.314899


### setting up the problem

In [22]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


PandasGUI ERROR — pandasgui.store — "None of [Index([1536899, 1536914, 1536916, 1536916, 1536979, 1536916, 1536911, 1536814,\n       1536814, 1536899,\n       ...\n       2044947, 2053434, 2050779, 2042929, 2044390, 2049176, 2054456, 2048705,\n       2048790, 2050778],\n      dtype='int64', length=3894)] are in the [index]"
Traceback (most recent call last):
  File "C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandasgui\store.py", line 621, in apply_filters
    df = df.query(filt.expr, engine='python')
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\frame.py", line 4453, in query
    result = self.loc[res]
             ~~~~~~~~^^^^^
  File "C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\

observations 
1. absorbed cause multicollinearity issues in the vars, be careful when using it. 

In [259]:
char_vars = new_x_names.copy()
char_vars.remove('x6')
prod_vars = list(dummies.columns) #dummies of product category 



formulation_vars = char_vars + prod_vars
#formulation_vars.remove('x5')
#formulation_vars.remove('x4')

formulation_string2 = '1 + ' + ' + '.join(formulation_vars) + ' + prices'

X1_formulation = pyblp.Formulation(formulation_string2, absorb = 'C(x5) + C(x4)') #absorb the categorical variables.
X1_formulation = pyblp.Formulation(formulation_string2) #absorb the categorical variables. 

X2_formulation = pyblp.Formulation('1 + ' + ' + '.join(prod_vars) + ' + prices') #X2: non-linear part does not allow absorb

product_formulations = (X1_formulation, X2_formulation ) 



product_formulations

(1 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + prices,
 1 + x6 + x7 + x8 + x9 + x10 + prices)

In [260]:
agent_formulation = pyblp.Formulation('1 +' + agent_formulation_string)
agent_formulation

problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data, costs_type='log')
problem


Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N      I     K1    K2    D    MD 
---  ----  -----  ----  ----  ---  ----
217  2936  52876   19    7     8    18 

Formulations:
       Column Indices:          0                1                           2                           3                 4          5              6                       7                          8                    9              10                        11                               12                   13        14        15        16        17        18  
-----------------------------  ---  ---------------------------  --------------------------  -------------------------  --------  ---------  ------------------  -------------------------  --------------------------  ------------  --------------  ----------------------------------  -----------------------------  --------  --------  --------  --------  ---------  ------
 X1: Linear Characteristics     1   x1['d

Detected collinearity issues with [x4['no tiene'], x5['antideslumbrante manual'], x6[True]] and at least one other column in X1. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.
Detected collinearity issues with [x4['no tiene'], x5['antideslumbrante manual'], x6[True]] and at least one other column in ZD. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.


Dimensions:
 T    N      I     K1    K2    D    MD 
---  ----  -----  ----  ----  ---  ----
217  2936  52876   19    7     8    18 

Formulations:
       Column Indices:          0                1                           2                           3                 4          5              6                       7                          8                    9              10                        11                               12                   13        14        15        16        17        18  
-----------------------------  ---  ---------------------------  --------------------------  -------------------------  --------  ---------  ------------------  -------------------------  --------------------------  ------------  --------------  ----------------------------------  -----------------------------  --------  --------  --------  --------  ---------  ------
 X1: Linear Characteristics     1   x1['delantero - delantera']  x1['delantero - integral']  x1['delantero - tr

### Setting up micro moments

Create the observed moments composed by 
1. $$  G^2(\theta) = Pr(\{\text{i purchases }c\}| i\in k), \ c\in \{\text{SUV, Pick-up}\}  $$ 
2. $$     G^3(\theta) = \mathbb{E}[p_{jt}| i\in k]
$$

In [261]:
micro_dataset = pyblp.MicroDataset(
    name="transactions",
    observations= len(agent_data),
    compute_weights=lambda t, p, a: np.ones((a.size, 1 + p.size)), #define a function that computes the weights and will be used later 
)
micro_dataset

transactions: 52876 Observations in All Markets

In [262]:
## 1. create the first set of micro moments (G2) 
avg_outside_share = product_data[product_data['product_ids'] == 0]['shares'].mean() #average share of outside good
 

micro_statistics = df.groupby(['k', 'x6']).size().reset_index(name='count') #count of each k, x6 combination where x6 is the product category
micro_statistics['N'] = df['k'].value_counts()[micro_statistics['k']].values
micro_statistics['share'] = micro_statistics['count'] / micro_statistics['N']
micro_statistics['share'] = micro_statistics['share']/(1 + avg_outside_share) #normalize by the outside share

for i, k_value in enumerate(unique_k, 1):    
    micro_statistics[f'a{i}'] = (micro_statistics['k'] == k_value).astype(int)

# Generate dummy variables
dummies = pd.get_dummies(micro_statistics['x6'])
micro_statistics.rename(columns={'x6': 'cat_var'}, inplace=True)
dummies.columns = [f'x{i}' for i in range(6, 6 + dummies.shape[1])]
micro_statistics = pd.concat([micro_statistics, dummies], axis=1)


## 2. create the second set of micro moments (G3)
avg_prices = df.groupby('k')['Precio Unitario'].mean()
print(avg_prices)


for k, price in avg_prices.items():
    micro_statistics.loc[micro_statistics['k'] == k, 'prices'] = price
    micro_statistics = pd.concat([micro_statistics, pd.DataFrame({'k': [k], 'prices': [price]})], ignore_index=True)

k
0    1.804686e+07
1    1.667907e+07
2    1.973762e+07
3    1.644347e+07
4    1.770722e+07
5    1.770260e+07
6    1.752879e+07
Name: Precio Unitario, dtype: float64


In [263]:
x_list = [f'x{i}' for i in range(6, 11)]
print(x_list)

a_list = [f'a{i}' for i in range(1, len(unique_k))]
print(a_list)

['x6', 'x7', 'x8', 'x9', 'x10']
['a1', 'a2', 'a3', 'a4', 'a5', 'a6']


In [264]:
inside_micro_parts = {}
inside_micro_price = {} # dictionary to store the numerator of the price moments
micro_parts = {}

for num, j in enumerate(a_list, 1): 
    micro_parts[f'{j}=1'] = pyblp.MicroPart( #copying mid_part
        name = f'E[{j}=1]',
        dataset=micro_dataset,
        compute_values=lambda t, p, a: np.outer(a.demographics[:, num], np.r_[0, p.X2[:, 0]]), 
        #in mid_part they use X2, 
    )

    inside_micro_price[f'E[price*{j}=1]'] = pyblp.MicroPart( #copying inside_mid_part 
        name=f'E[{j}=1*price]',
        dataset=micro_dataset,
        compute_values=lambda t, p, a: np.outer(a.demographics[:, num], np.r_[1, p.X2[:, 3]]), 
        )
    
    for num2, i in enumerate(x_list, 1): 
        inside_micro_parts[f'E[{j}=1*{i}=1]'] = pyblp.MicroPart( #copying inside_mid_part 
            name=f'E[{j}=1*{i}=1]',
            dataset=micro_dataset,
            compute_values=lambda t, p, a: np.outer(a.demographics[:, num], np.r_[1, p.X2[:, num2]]), 
        )

In [265]:
compute_ratio = lambda v: v[0] / v[1]
compute_ratio_gradient = lambda v: [1 / v[1], -v[0] / v[1]**2]

micro_moments = []

for num, j  in enumerate(a_list, 1): 
    value = micro_statistics.loc[ (micro_statistics[j] == 1), 'prices'].iloc[0]
    micro_moments.append(
        pyblp.MicroMoment(
            name=f'E[prices|{j}=1]',
            value = value,
            parts = [inside_micro_price[f'E[price*{j}=1]'], micro_parts[f'{j}=1']],
            compute_value=compute_ratio,
            compute_gradient=compute_ratio_gradient,
        )
    )
    
    for num2, i in enumerate(x_list, 1): 
        try: 
            value = micro_statistics.loc[(micro_statistics[j] == 1) & (micro_statistics[i] == 1), 'share'].iloc[0]
            micro_moments.append(
                pyblp.MicroMoment(
                    name=f'E[{i}=1|{j}=1]',
                    value=value, 
                    parts= [inside_micro_parts[f'E[{j}=1*{i}=1]'], micro_parts[f'{j}=1'] ], 
                    compute_value=compute_ratio,
                    compute_gradient=compute_ratio_gradient,
                )
            )
            print('works', j, i)
        except: 
            continue
        

works a1 x6
works a1 x7
works a1 x8
works a1 x9
works a2 x6
works a2 x7
works a2 x8
works a2 x9
works a3 x6
works a3 x7
works a3 x8
works a3 x9
works a4 x6
works a4 x7
works a4 x8
works a4 x9
works a5 x6
works a5 x7
works a5 x8
works a5 x9
works a6 x6
works a6 x7
works a6 x8
works a6 x9


In [241]:
print(len(micro_moments))
print(micro_moments)

30
[E[prices|a1=1]: +1.804686E+07 (E[a1=1*price] on transactions: 52876 Observations in All Markets; E[a1=1] on transactions: 52876 Observations in All Markets), E[x6=1|a1=1]: +3.769602E-01 (E[a1=1*x6=1] on transactions: 52876 Observations in All Markets; E[a1=1] on transactions: 52876 Observations in All Markets), E[x7=1|a1=1]: +8.512004E-02 (E[a1=1*x7=1] on transactions: 52876 Observations in All Markets; E[a1=1] on transactions: 52876 Observations in All Markets), E[x8=1|a1=1]: +7.782404E-02 (E[a1=1*x8=1] on transactions: 52876 Observations in All Markets; E[a1=1] on transactions: 52876 Observations in All Markets), E[x9=1|a1=1]: +1.434881E-01 (E[a1=1*x9=1] on transactions: 52876 Observations in All Markets; E[a1=1] on transactions: 52876 Observations in All Markets), E[prices|a2=1]: +1.667907E+07 (E[a2=1*price] on transactions: 52876 Observations in All Markets; E[a2=1] on transactions: 52876 Observations in All Markets), E[x6=1|a2=1]: +3.658197E-01 (E[a2=1*x6=1] on transactions: 5

In [1]:
print(unique_k)

NameError: name 'unique_k' is not defined

### solving the problem

In [273]:
dim_X2 = 7

initial_sigma = np.eye(dim_X2) # square matrix with the dimension of X2 
initial_pi = np.array(np.ones((dim_X2, 1+len(unique_k)))) # dim(x2) x dim(agent formulation)


# We'll build the sigma_bounds so that off-diagonals are exactly 0
sigma_bounds = []
for i in range(dim_X2):
    for j in range(i + 1):
        if i == j:
            # Diagonal: allow > 0, no upper bound
            sigma_bounds.append((0.0, None))
        else:
            # Off-diagonal forced to 0
            sigma_bounds.append((0.0, 0.0))


# Build full k x k arrays for lb and ub
lb_sigma = np.zeros((dim_X2, dim_X2))
ub_sigma = np.zeros((dim_X2, dim_X2))

for i in range(dim_X2):
    for j in range(dim_X2):
        if i == j:
            # Diagonal: allow >= 0, no upper bound.
            lb_sigma[i, j] = 0.0
            # pyblp treats None as +∞
            ub_sigma[i, j] = None
        else:
            # Off-diagonal forced to 0
            lb_sigma[i, j] = 0.0
            ub_sigma[i, j] = 0.0

sigma_bounds = (lb_sigma, ub_sigma)


In [275]:
results = problem.solve(
    sigma=initial_sigma,
    sigma_bounds  = sigma_bounds, 
    pi=initial_pi,
    optimization=pyblp.Optimization('bfgs', {'gtol': 1e-4}),
    iteration=pyblp.Iteration('squarem', {'atol': 1e-13}),
    #se_type='clustered',
    #W_type='clustered',
    micro_moments=micro_moments,
)
results

Solving the problem ...

Micro Moments:
  Observed         Moment          Part         Dataset     Observations  Markets
-------------  --------------  -------------  ------------  ------------  -------
+1.804686E+07  E[prices|a1=1]  E[a1=1*price]  transactions     52876        All  
                                  E[a1=1]     transactions     52876        All  
+3.769602E-01   E[x6=1|a1=1]   E[a1=1*x6=1]   transactions     52876        All  
                                  E[a1=1]     transactions     52876        All  
+8.512004E-02   E[x7=1|a1=1]   E[a1=1*x7=1]   transactions     52876        All  
                                  E[a1=1]     transactions     52876        All  
+7.782404E-02   E[x8=1|a1=1]   E[a1=1*x8=1]   transactions     52876        All  
                                  E[a1=1]     transactions     52876        All  
+1.434881E-01   E[x9=1|a1=1]   E[a1=1*x9=1]   transactions     52876        All  
                                  E[a1=1]     transactions

The model may be under-identified. The total number of unfixed parameters is 82, which is more than the total number of moments, 48. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.
Detected that the 2SLS weighting matrix is nearly singular with condition number +3.391465E+24. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of micro moments is nearly singular with condition number +3.961952E+161. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +5.672261E+22. To disable singularity checks, set options.singular_tol = numpy.inf.


Computed results after 00:00:52.

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

Detected that the estimated covariance matrix of micro moments is nearly singular with condition number +3.961952E+161. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +2.923693E+24. To disable singularity checks, set options.singular_tol = numpy.inf.


Computed results after 01:01:12.

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number
----  -------------  -------------  --------------  --------------  -------  ----------------
 1    +1.605352E+02  +0.000000E+00  +0.000000E+00   +0.000000E+00    2710         +INF       

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                                                                                                                                                                                                                                                                                                                                                                                                                                

Detected that the estimated covariance matrix of micro moments is nearly singular with condition number +3.961952E+161. To disable singularity checks, set options.singular_tol = numpy.inf.


Computed results after 01:19:09.

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +1.990004E+02  +0.000000E+00  +0.000000E+00   +0.000000E+00    2710         +INF               +INF       

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 02:22:14       Yes          0             5          6304         19780   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
 Sigma:           1            x6[True]         x7[True]         x8[True]         x9[True]         

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.812110E+23. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +2.303952E+24. To disable singularity checks, set options.singular_tol = numpy.inf.


Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +1.990004E+02  +0.000000E+00  +0.000000E+00   +0.000000E+00    2710         +INF               +INF       

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 02:22:14       Yes          0             5          6304         19780   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
 Sigma:           1            x6[True]         x7[True]         x8[True]         x9[True]         x10[True]         prices       |  